<a href="https://colab.research.google.com/github/eeolga/article/blob/main/Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initial Data Processing and User Grouping


Load the initial raw data, extract user information, and group events log by user to calculate 'Count of Events'.


In [ ]:
import pandas as pd
import re

# Download file '/content/ecourse-1.xlsx' (resource Assessment logs)
# The file has an .xlsx extension, so pd.read_excel is the correct function to use.
df = pd.read_excel('/content/131-3.xlsx')

# Function to extract the first number from a string
def get_first_number(text):
    numbers = re.findall(r'\d+', str(text))
    return numbers[0] if numbers else None

# Function to extract the last number from a string
def get_last_number(text):
    numbers = re.findall(r'\d+', str(text))
    return numbers[-1] if numbers else None

# Apply the functions to update the specified columns
df['User full name'] = df['Description'].apply(get_first_number)

# compute total events per user
# Group by 'User full name' and count 'Sündmuse kontekst' to get the number of events
user_totals = df.groupby("User full name")["Event context"].count().reset_index()
user_totals.rename(columns={'Event context': 'Count of Events'}, inplace=True);

# Save the aggregated DataFrame to '1.xlsx'
user_totals.to_excel('1.xlsx', index=False)

## Calculate Time Metrics and WTglobal


Using the 'user_totals' DataFrame from the previous step, calculate 'Total Time (seconds)', 'Total Time (hours)', and 'WTglobal' for each user.


In [ ]:
import pandas as pd

# Assuming user_totals DataFrame is already available from previous steps
# If not, you would load it from '1.xlsx':
user_totals = pd.read_excel('1.xlsx')

# Define the constants from the formula
pa = 0.7
i_bar = 120 # seconds

# Calculate seconds per event
seconds_per_event = pa * i_bar

# Calculate 'Total Time (seconds)' per user
user_totals['Total Time (seconds)'] = user_totals['Count of Events'] * seconds_per_event

# Convert to hours for readability
user_totals['Total Time (hours)'] = user_totals['Total Time (seconds)'] / 3600 # Corrected to divide by 3600 for hours

# Remove 'Total Time (minutes)' column if it exists
user_totals = user_totals.drop(columns=['Total Time (minutes)'], errors='ignore')

# Define the constants for WTglobal calculation
# ∑resource = 3 * 26 academic hours
# 1 academic hour = 45 minutes = 45 * 60 seconds = 2700 seconds
sum_resource_academic_hours = 3 * 26
sum_resource_seconds = sum_resource_academic_hours * 45 * 60 # Convert to seconds

# Use the Ij of e-course from metamodel for Parameter_global calculation
Ij = 0.0049033

# Calculate WTglobal for each user
# Tr = user_totals['Total Time (seconds)']
user_totals['WTglobal'] = (user_totals['Total Time (seconds)'] / sum_resource_seconds) * Ij

# Save the updated DataFrame to '2.xlsx'
user_totals.to_excel('2.xlsx', index=False)

print("WTglobal calculated and saved to '2.xlsx'.")

# Display the first few rows of the updated DataFrame
display(user_totals.head())

WTglobal calculated and saved to '2.xlsx'.


,User full name,Count of Events,Total Time (seconds),Total Time (hours),WTglobal
0,29213,17,1428.0,0.396667,0.000033
1,32686,13,1092.0,0.303333,0.000025
2,32760,15,1260.0,0.350000,0.000029
3,32926,38,3192.0,0.886667,0.000074
4,32962,13,1092.0,0.303333,0.000025


## Calculate WLglobal,i


Further update the 'user_totals' DataFrame by calculating 'WLresource,i' and 'WLglobal,i' based on the specified formulas and metamodel data.


In [ ]:
# Load the 'user_totals' DataFrame if not already in memory (from '2.xlsx')
# This step is important to ensure the latest data is used if the kernel resets or this cell is run independently.
user_totals = pd.read_excel('2.xlsx')

# 1. Calculate ∑Li (sum of all resource logs, which is the total count of events across all users)
sum_Li = user_totals['Count of Events'].sum()

# Define I,j of Assessment with deadline
I_j = 0.0049033

# 2. Calculate WLresource,i for each user (based on the previous definition: Logs,user / sum_Li)
# Note: This column might not be directly used in the WLglobal,i formula if WLresource,i is defined as raw logs.
user_totals['WLresource,i'] = user_totals['Count of Events'] / sum_Li

# 3. Calculate WLglobal,i using the corrected formula: (WLresource,i * I,j) / (∑Li)
# Where WLresource,i is now interpreted as 'Logs,user' (user_totals['Count of Events']) for this specific formula.
user_totals['WLglobal,i'] = (user_totals['Count of Events'] * I_j) / sum_Li

# Save the updated DataFrame to '3.xlsx'
user_totals.to_excel('3.xlsx', index=False)

print("WLglobal,i calculated and saved to '3.xlsx'.")

# Display the first few rows of the updated DataFrame
display(user_totals.head())

WLglobal,i calculated and saved to '3.xlsx'.


,User full name,Count of Events,Total Time (seconds),Total Time (hours),WTglobal,"WLresource,i","WLglobal,i"
0,29213,17,1428,0.396667,0.000033,0.075221,0.000369
1,32686,13,1092,0.303333,0.000025,0.057522,0.000282
2,32760,15,1260,0.350000,0.000029,0.066372,0.000325
3,32926,38,3192,0.886667,0.000074,0.168142,0.000824
4,32962,13,1092,0.303333,0.000025,0.057522,0.000282


## Calculate S_norm,i and Final Save


Perform the final calculations for 'S_resource,i' and 'S_norm,i'. After all calculations are complete, save the fully processed 'user_totals' DataFrame to a single final output file (e.g., 'final_results.xlsx'), eliminating the need for multiple intermediate files.


In [ ]:
import pandas as pd

# Load the 'user_totals' DataFrame from '3.xlsx' to get the latest calculations
user_totals = pd.read_excel('3.xlsx')

# Define the normalization factors
alpha = 0.2
beta = 0.3
gamma = 0.5

# Define the constant WDglobal,i from metamodel data of the Assessment with deadline
WDglobal_i = 0.00212266

# Calculate the numerator (α∙WLglobal,i + β∙WDglobal,i + γ∙WTglobal,i)
# This is interpreted as S_resource,i in the context of the denominator sum.
user_totals['S_resource,i'] = (alpha * user_totals['WLglobal,i']) + \
                               (beta * WDglobal_i) + \
                               (gamma * user_totals['WTglobal'])

# Calculate the sum of S_resource,i across all users for the denominator
sum_S_resource = user_totals['S_resource,i'].sum()

# Calculate S_norm,i
user_totals['S_norm,i'] = user_totals['S_resource,i'] / sum_S_resource

# Save the final DataFrame to 'final_results.xlsx'
user_totals.to_excel('4.xlsx', index=False)

print("S_norm,i calculated and saved to '4.xlsx'.")

# Display the first few rows of the updated DataFrame
display(user_totals.head())

S_norm,i calculated and saved to '4.xlsx'.


,User full name,Count of Events,Total Time (seconds),Total Time (hours),WTglobal,"WLresource,i","WLglobal,i","S_resource,i","S_norm,i"
0,29213,17,1428,0.396667,0.000033,0.075221,0.000369,0.000727,0.057422
1,32686,13,1092,0.303333,0.000025,0.057522,0.000282,0.000706,0.055742
2,32760,15,1260,0.350000,0.000029,0.066372,0.000325,0.000717,0.056582
3,32926,38,3192,0.886667,0.000074,0.168142,0.000824,0.000839,0.066239
4,32962,13,1092,0.303333,0.000025,0.057522,0.000282,0.000706,0.055742


In [ ]:
import numpy as np

# Define global parameters (these values are already present in user_totals but explicit definition can be useful)
# Note: Sglobal, WTglobal, WLglobal, WDglobal are not single global values but rather columns or derived values per user.
# The task refers to these as 'global parameters', which in this context means utilizing the columns already calculated.

# Calculate the 25th percentile of S_norm,i
percentile_25 = user_totals['S_norm,i'].quantile(0.25)

# Identify 'At_Risk' students as 'Yes' or 'No'
user_totals['At_Risk'] = np.where(user_totals['S_norm,i'] < percentile_25, 'Yes', 'No')

# Save the updated DataFrame to '5.xlsx'
user_totals.to_excel('final.xlsx', index=False)

print(f"25th percentile of S_norm,i: {percentile_25:.6f}")
print(f"Number of At_Risk students: {user_totals[user_totals['At_Risk'] == 'Yes'].shape[0]}")
print("DataFrame updated with 'At_Risk' column and saved to 'final.xlsx'.")

display(user_totals.head())

25th percentile of S_norm,i: 0.052803
Number of At_Risk students: 4
DataFrame updated with 'At_Risk' column and saved to 'final.xlsx'.


,User full name,Count of Events,Total Time (seconds),Total Time (hours),WTglobal,"WLresource,i","WLglobal,i","S_resource,i","S_norm,i",At_Risk
0,29213,17,1428,0.396667,0.000033,0.075221,0.000369,0.000727,0.057422,No
1,32686,13,1092,0.303333,0.000025,0.057522,0.000282,0.000706,0.055742,No
2,32760,15,1260,0.350000,0.000029,0.066372,0.000325,0.000717,0.056582,No
3,32926,38,3192,0.886667,0.000074,0.168142,0.000824,0.000839,0.066239,No
4,32962,13,1092,0.303333,0.000025,0.057522,0.000282,0.000706,0.055742,No
